In [ ]:
import psycopg2  # PostgreSQL database adapter for Python
import pandas as pd  # Library for data manipulation and analysis
from sqlalchemy import create_engine  # SQLAlchemy engine for database interaction
from psycopg2 import sql  # SQL string composition for psycopg2
from selenium import webdriver  # WebDriver for web scraping
from selenium.webdriver.common.by import By  # Selenium locators
from selenium.webdriver.support.ui import WebDriverWait  # WebDriverWait for explicit waits
from selenium.webdriver.support import expected_conditions as EC  # Expected conditions for waits
from selenium.common.exceptions import TimeoutException, WebDriverException  # Exception handling for Selenium
import time  # Time module for delays

# Database connection details
db_user = 'postgres'
db_password = 'password'
db_host = 'localhost'
db_port = '5432'
db_name = 'bus_details'

# Function to connect to the PostgreSQL database
def connect_db():
    return psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )

# Initialize WebDriver
driver = webdriver.Chrome()

# List of websites to scrape
websites = [
    'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',
    'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/kaac-transport',
    'https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/jksrtc'
]

# Lists to store route names and links
route_names = []
route_links = []

# Scraping route data
for website in websites:
    driver.get(website)
    time.sleep(2)  # Wait for the page to load

    try:
        # Find the pagination table
        pagination_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
        pages = pagination_table.find_elements(By.XPATH, './/div[contains(@class, "pageTabs")]')

        # Determine the last page number
        if pages:
            last_page = int(pages[-1].text)
        else:
            last_page = 1

        current_page = 1

        while current_page <= last_page:
            try:
                # Wait for the container element to be visible
                container = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'route_link')))
                route_name_elements = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'route')))

                # Collect route names and links
                for route in route_name_elements:
                    route_names.append(route.text)
                    route_links.append(route.get_attribute('href'))

                # Move to the next page
                current_page += 1
                next_page = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f'//div[contains(@class, "pageTabs") and text()="{current_page}"]')))

                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(2)  # Wait for the next page to load
            except Exception as e:
                print(f"Exception during pagination: {str(e)}")
                break
    except TimeoutException:
        print(f"Timeout while accessing pagination for {website}")
        continue

# Create a DataFrame from the collected route names and links
df_route = pd.DataFrame({'route_name': route_names, 'route_link': route_links})

# Save DataFrame to CSV
df_route.to_csv('route_data.csv', index=False)

# Lists to store bus details
bus_names = []
bus_types = []
departing_times = []
durations = []
reaching_times = []
star_ratings = []
prices = []
seat_availabilities = []
bus_route_names = []
bus_route_links = []

# Collect bus data for each route link
for route_name, link in zip(route_names, route_links):
    try:
        driver.get(link)
        time.sleep(5)  # Wait for the page to load
        driver.execute_script("window.scrollBy(0, 200);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(8)  # Wait for dynamic content to load

        # Find bus detail containers
        containers = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix bus-item-details"]')))

        # Extract details from each container
        for container in containers:
            try:
                bus_name = container.find_element(By.CLASS_NAME, 'travels.lh-24.f-bold.d-color').text
            except:
                bus_name = 'N/A'
            try:
                bus_type = container.find_element(By.CLASS_NAME, 'bus-type.f-12.m-top-16.l-color.evBus').text
            except:
                bus_type = 'N/A'
            try:
                departing = container.find_element(By.CLASS_NAME, 'dp-time.f-19.d-color.f-bold').text
            except:
                departing = 'N/A'
            try:
                total_duration = container.find_element(By.CLASS_NAME, 'dur.l-color.lh-24').text
            except:
                total_duration = 'N/A'
            try:
                reaching = container.find_element(By.CLASS_NAME, 'bp-time.f-19.d-color.disp-Inline').text
            except:
                reaching = 'N/A'
            try:
                star = container.find_element(By.XPATH, './/div[5]/div[1]/div/span').text
            except:
                star = 'N/A'
            try:
                ticket_price = container.find_element(By.XPATH, './/*[contains(@class, "fare d-block")]/span').text
            except:
                ticket_price = 'N/A'
            try:
                seat = container.find_element(By.XPATH, './/div[contains(@class, "seat-left m-top-30")]').text
            except:
                seat = 'N/A'

            # Append collected data to respective lists
            bus_names.append(bus_name)
            bus_types.append(bus_type)
            departing_times.append(departing)
            durations.append(total_duration)
            reaching_times.append(reaching)
            star_ratings.append(star)
            prices.append(ticket_price)
            seat_availabilities.append(seat)
            bus_route_names.append(route_name)
            bus_route_links.append(link)

    except TimeoutException:
        continue
    except WebDriverException as e:
        print(f"WebDriverException occurred: {str(e)}")
        continue

# Create a DataFrame from the collected bus details
df_bus = pd.DataFrame({
    'route_name': bus_route_names,
    'route_link': bus_route_links,
    'bus_name': bus_names,
    'bus_type': bus_types,
    'departing_time': departing_times,
    'duration': durations,
    'reaching_time': reaching_times,
    'star_rating': star_ratings,
    'price': prices,
    'seat_availability': seat_availabilities
})

# Save DataFrame to CSV
df_bus.to_csv('bus_data.csv', index=False)

# Close the WebDriver
driver.quit()

print("Data saved to bus_data.csv.")

# Connect to PostgreSQL to create the database
conn = psycopg2.connect(
    dbname='postgres',
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

conn.autocommit = True  # Enable autocommit mode
cur = conn.cursor()

# Create database
try:
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
    print(f"Database {db_name} created successfully")
except psycopg2.errors.DuplicateDatabase:
    print(f"Database {db_name} already exists")

cur.close()
conn.close()

# Connect to the new database
conn = connect_db()
cur = conn.cursor()

# Create table
create_bus_table = """
CREATE TABLE IF NOT EXISTS bus_data (
    id SERIAL PRIMARY KEY,
    route_name TEXT,
    route_link TEXT,
    bus_name TEXT,
    bus_type TEXT,
    departing_time TEXT,
    duration TEXT,
    reaching_time TEXT,
    star_rating FLOAT,
    price TEXT,
    seat_availability TEXT
);
"""

try:
    cur.execute(create_bus_table)
    conn.commit()
    print("Table created successfully")
except Exception as e:
    print(f"Error creating table: {e}")
    conn.rollback()

cur.close()
conn.close()

# Create a connection string for SQLAlchemy
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Load data from CSV files
df_bus = pd.read_csv('bus_data.csv')

# Insert data into PostgreSQL table
try:
    df_bus.to_sql('bus_data', engine, index=False, if_exists='replace')
    print("Data loaded into PostgreSQL successfully!")
except Exception as e:
    print(f"Error loading data: {e}")
